In [1]:
#devtools::install_url('https://github.com/catboost/catboost/releases/download/v1.0.6/catboost-R-Linux-1.0.6.tgz', INSTALL_opts = c("--no-multiarch", "--no-test-load"))

In [6]:
source('https://raw.githubusercontent.com/sametsoekel/nonsystematic_workspace/master/space/cb_pipeline0816.r')

Warning message:
"package 'caret' was built under R version 4.2.1"


In [7]:
config <- list(
    label = 'default',
    seed = 1453,
    sparse_percentage = 0.003,
    folds = 3,
    learning_rate = 0.01,
    early_stopping_rounds = 200,
    iter = 10000,
    objective = 'Logloss',
    loss_fun = 'F1:use_weights=False',
    eval_metric = 'F1:use_weights=False',
    auto_class_weights = 'SqrtBalanced',
    use_best = TRUE,
    task = 'CPU',
    device = '0:1',
    slicer_1 = '*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*',
    slicer_2 = '_____________________________________________'
)

In [8]:
data_sl <- data.table()

for(i in c(LETTERS[1:10],letters[1:20])){
    charmi <- sample(c(TRUE,FALSE),size = 1)
    ortsec <- sample(200:500,size = 1)
    sdsec <- sample(100:1000,size = 1)
    if(charmi){
        data_sl[,(i) := sample(c(LETTERS,NA),size = 1.7e6,replace = T)]
    }else{
        data_sl[,(i) := sample(c(rnorm(n = 2e6,mean = ortsec,sd = sdsec),NA),size = 1.7e6,replace = T)]
    }
}

data_sl[,default := sample(c('No','Yes'),size = 1.7e6,prob = c(.93,.07),replace = T)]

In [9]:
set.seed(571)
traindex <- createDataPartition(data_sl[['default']],p = .8,times = 1,list = FALSE) %>% as.numeric
train_set <- data_sl[traindex,]
test_set <- data_sl[-traindex,]

In [10]:
train_ok <- train_set %>% make_frame(config = config)
test_ok <- test_set %>% select(-default) %>% make_frame(test = T,train_frame = train_ok,config = config)

Data sparsed.

Scaling factors calculated.

Missing values labeled.

Data scaled.

Got distinct values for nominals.

Data dummied.

All done !

Data sparsed.

Scaling factors calculated.

Missing values labeled.

Data scaled.

Unseen values removed from test set.

Train & test set nominal levels fetched.

Data dummied.

All done !



In [11]:
enc <- function(x){
    case_when(x == 'No' ~ 0,
              x == 'Yes' ~ 1)
}

encback <- function(x){
    case_when(x == 1 ~ 'Yes',
              x == 0 ~ 'No')
}

In [12]:
fld <- create_folds(train_ok,config = config)
cved <- cross_validate(train_ok,test_ok,folds = fld,config = config,label_encoder = enc,label_back_encoder =encback)

[1] "*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*"

[1] "Fold 1 F1 : 0.131"

[1] "Best threshold for Fold 1 : 0.1"

[1] "_____________________________________________"

[1] "Elapsed time for Fold 1 : 5.3342 minutes"

...

[1] "Best threshold for Fold 3 : 0.1"

[1] "_____________________________________________"

[1] "Elapsed time for Fold 3 : 4.2596 minutes"

[1] "*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*"

[1] "CV Mean F1 : 0.131"

In [ ]:
kk <- df %>% rowwise %>% mutate(pred = fmode(Fold1,Fold2,Fold3)) %>% as.data.table

In [ ]:
obs <- test_set %>% select(default) %>% pull %>% as.factor

In [ ]:
prd <- kk %>% select(pred) %>% pull %>% as.factor

In [ ]:
f_meas_vec(truth = obs,estimate = factor(prd,levels=c('No','Yes')),event_level = 'second')